In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras import callbacks
import numpy as np
import pickle
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model, load_model

Using TensorFlow backend.


In [2]:
model = load_model('../models/seq2seq_model_3layers.hdf5')
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, None, 3125)   0                                            
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     (None, None, 3130)   0                                            
__________________________________________________________________________________________________
encoder_lstm1 (LSTM)            [(None, None, 256),  3463168     encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_lstm1 (LSTM)            [(None, None, 256),  3468288     decoder_inputs[0][0]             
                                                                 encoder_lstm1[0][1]        

### 构造inference的encoder模型（自己构造input，放到使用训练好的模型中进行计算，得到output）

In [3]:
encoder_lstm1 = model.get_layer("encoder_lstm1")
encoder_lstm2 = model.get_layer("encoder_lstm2")
encoder_lstm3 = model.get_layer("encoder_lstm3")

encoder_inputs = Input(shape=(None, 3125), name='encoder_input')

encoder_outputs1, state_h1, state_c1 = encoder_lstm1(encoder_inputs)
encoder_outputs2, state_h2, state_c2 = encoder_lstm2(encoder_outputs1)
encoder_outputs3, state_h3, state_c3 = encoder_lstm3(encoder_outputs2)

encoder_states = [state_h1, state_c1, state_h2, state_c2, state_h3, state_c3]
encoder_model = Model(encoder_inputs, encoder_states)

In [4]:
encoder_model.output

[<tf.Tensor 'encoder_lstm1_1/while:4' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'encoder_lstm1_1/while:5' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'encoder_lstm2_1/while:4' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'encoder_lstm2_1/while:5' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'encoder_lstm3_1/while:4' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'encoder_lstm3_1/while:5' shape=(None, 256) dtype=float32>]

### 构造inference的decoder模型（自己构造input，放到使用训练好的模型中进行计算，得到output）

In [5]:
decoder_lstm1 = model.get_layer("decoder_lstm1")
decoder_lstm2 = model.get_layer("decoder_lstm2")
decoder_lstm3 = model.get_layer("decoder_lstm3")
decoder_dense = model.get_layer("dense_1")

decoder_inputs = Input(shape=(None, 3130), name='decoder_input')
decoder_state_input_h1 = Input(shape=(256,))
decoder_state_input_c1 = Input(shape=(256,))
decoder_states_inputs1 = [decoder_state_input_h1, decoder_state_input_c1]

decoder_state_input_h2 = Input(shape=(256,))
decoder_state_input_c2 = Input(shape=(256,))
decoder_states_inputs2 = [decoder_state_input_h2, decoder_state_input_c2]

decoder_state_input_h3 = Input(shape=(256,))
decoder_state_input_c3 = Input(shape=(256,))
decoder_states_inputs3 = [decoder_state_input_h3, decoder_state_input_c3]

decoder_outputs1, state_h1, state_c1 = decoder_lstm1(decoder_inputs, initial_state=decoder_states_inputs1)
decoder_outputs2, state_h2, state_c2 = decoder_lstm2(decoder_outputs1, initial_state=decoder_states_inputs2)
decoder_outputs3, state_h3, state_c3 = decoder_lstm3(decoder_outputs2, initial_state=decoder_states_inputs3)

decoder_states = [state_h1, state_c1, state_h2, state_c2, state_h3, state_c3]
decoder_outputs = decoder_dense(decoder_outputs3)
decoder_states_inputs = decoder_states_inputs1 + decoder_states_inputs2 + decoder_states_inputs3

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [6]:
decoder_model.output

[<tf.Tensor 'dense_1_1/truediv:0' shape=(None, None, 3125) dtype=float32>,
 <tf.Tensor 'decoder_lstm1_1/while:4' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'decoder_lstm1_1/while:5' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'decoder_lstm2_1/while:4' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'decoder_lstm2_1/while:5' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'decoder_lstm3_1/while:4' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'decoder_lstm3_1/while:5' shape=(None, 256) dtype=float32>]

In [8]:
encoder_model.save("../models/encoder_model.hdf5")
decoder_model.save("../models/decoder_model.hdf5")